Notebook with functions for plotting anomalies and raw time series.

# Ocean scalar variables

## Regular anomaly and mean

In [1]:
def plot_ts_diff(diff_type,fig_dir,start_year,end_year,anom_var="thetaoga",title_suff="Volume Mean Ocean Temperature Anomaly",fig_pref=None,
                 ylimits = [-0.1,1.2],
                 leg_loc = 'upper left',
                 leg_ncols = 1,
                       profiles = ['surf','therm','mid','bot'], 
                       power_inputs = ['0.1TW', '0.2TW', '0.5TW'], 
                       power_var_suff = ['0p1TW', '0p2TW', '0p5TW'], 
                       power_strings = ['0.1 TW', '0.2 TW', '0.5 TW']):
    """
    Function to plot anomaly over time from time series data for a particular CO2 scenario (each power input on separate plot).
    
    Inputs:
    diff_type (str): one of
                    ['const-1860ctrl',
                    'doub-1860exp','doub-2xctrl','doub-1860ctrl',
                    'quad-1860exp','quad-4xctrl','quad-1860ctrl']
    fig_dir (str): directory path to save figure
    start_year (int): start year of avg period
    end_year (int): end year of avg period
    """

    prof_dict = {'surf': 'b',
                 'therm': 'm',
                 'mid': 'g',
                 'bot': 'r'}

    # separate plot for each power input
    for pow_idx, power in enumerate(power_var_suff):
        fig, ax = plt.subplots(figsize=(6,3))

        # Add a horizontal line at y=0
        ax.axhline(y=0, color='black', linestyle='-', linewidth=1)
        
        ds_root = f'{power}_{start_year}_{end_year}_diff'
        for prof in profiles:
            if diff_type == 'const-1860ctrl':
                ds_name = f'const_{prof}_{ds_root}'
            elif diff_type == 'doub-1860exp':
                ds_name = f'doub_{prof}_{ds_root}_1860'
            elif diff_type == 'doub-2xctrl':
                ds_name = f'doub_{prof}_{ds_root}_2xctrl'
            elif diff_type == 'doub-1860ctrl':
                ds_name = f'doub_{prof}_{ds_root}_const_ctrl'
            elif diff_type == 'quad-1860exp':
                ds_name = f'quad_{prof}_{ds_root}_1860'
            elif diff_type == 'quad-4xctrl':
                ds_name = f'quad_{prof}_{ds_root}_4xctrl'
            elif diff_type == 'quad-1860ctrl':
                ds_name = f'quad_{prof}_{ds_root}_const_ctrl'

            time = np.linspace(start_year,end_year,num=len(myVars[ds_name][anom_var]) )
            ax.plot(time,myVars[ds_name][anom_var],label=prof,color=prof_dict[prof])

        if diff_type == 'doub-1860exp' or diff_type == 'doub-1860ctrl':
            ctrl_diff_name = f'doub_ctrl_{start_year}_{end_year}_diff'
            time = np.linspace(start_year,end_year,num=len(myVars[ctrl_diff_name][anom_var]) )
            ax.plot(time,myVars[ctrl_diff_name][anom_var],label='control',color='k')

        ax.set_xlabel("Time (Years)")
        if anom_var == "ave_hfds":
            ax.set_ylabel("Flux Anomaly (W/m$^2$)")
        else:
            ax.set_ylabel("Temperature Anomaly ($\degree$C)")
            
        ax.set_xlim(start_year-1,end_year)
        ax.set_ylim(ylimits)
        ax.legend(loc=leg_loc,ncols=leg_ncols)
        ax.grid("both")
    
        ax.minorticks_on()
        ax.grid(which='major', linestyle='-', linewidth='0.5', color='gray')
            
        if diff_type == 'const-1860ctrl':
            title_str = f"Const CO2 {power_strings[pow_idx]} Cases\n"
            fig_name = f"const_{power}"
            
        elif diff_type == 'doub-1860exp':
            title_str = f"1pct2xCO2 Radiative {power_strings[pow_idx]} Cases\n"
            fig_name = f"2xCO2-const_{power}"
            
        elif diff_type == 'doub-2xctrl':
            title_str = f"1pct2xCO2 Mixing {power_strings[pow_idx]} Cases\n"
            fig_name = f"2xCO2-2xctrl_{power}"
            
        elif diff_type == 'doub-1860ctrl':
            title_str = f"1pct2xCO2 Total {power_strings[pow_idx]} Cases\n"
            fig_name = f"2xCO2-const-ctrl_{power}"
            
        elif diff_type == 'quad-1860exp':
            title_str = f"1pct4xCO2 Radiative {power_strings[pow_idx]} Cases\n"
            fig_name = f"4xCO2-const_{power}"
            
        elif diff_type == 'quad-4xctrl':
            title_str = f"1pct4xCO2 Mixing {power_strings[pow_idx]} Cases\n"
            fig_name = f"4xCO2-4xctrl_{power}"
            
        elif diff_type == 'quad-1860ctrl':
            title_str = f"1pct4xCO2 Total {power_strings[pow_idx]} Cases\n"
            fig_name = f"4xCO2-const-ctrl_{power}"

        ax.set_title(title_str+title_suff)
        
        if not os.path.exists(fig_dir):
            os.makedirs(fig_dir)

        if fig_pref != None:
            plt.savefig(fig_dir+f'{fig_pref}_{anom_var}_anom_{fig_name}_{start_year}_{end_year}.pdf', dpi=600, bbox_inches='tight')
        else:
            plt.savefig(fig_dir+f'{anom_var}_anom_{fig_name}_{start_year}_{end_year}.pdf', dpi=600, bbox_inches='tight')

In [2]:
def plot_ts_mean(co2_scen,fig_dir,start_year,end_year,var="thetaoga",title_suff="Volume Mean Ocean Temperature",fig_pref=None,
                 ylimits = [3.0,5.0],
                 leg_loc = 'upper left',
                 leg_ncols = 1,
                       profiles = ['surf','therm','mid','bot'], 
                       power_inputs = ['0.1TW', '0.2TW', '0.5TW'], 
                       power_var_suff = ['0p1TW', '0p2TW', '0p5TW'], 
                       power_strings = ['0.1 TW', '0.2 TW', '0.5 TW']):
    """
    Function to plot raw time series variable (not anomaly).
    
    Inputs:
    co2_scen (str): one of ['const','doub','quad']
    fig_dir (str): directory path to save figure
    start_year (int): start year of avg period
    end_year (int): end year of avg period
    """

    prof_dict = {'surf': 'b',
                 'therm': 'm',
                 'mid': 'g',
                 'bot': 'r'}

    # separate plot for each power input
    for pow_idx, power in enumerate(power_var_suff):
        fig, ax = plt.subplots(figsize=(6,3))

        # Add a horizontal line at y=0
        ax.axhline(y=0, color='black', linestyle='-', linewidth=1)
        
        ds_root = f'{power}_{start_year}_{end_year}'
        for prof in profiles:
            ds_name = f'{co2_scen}_{prof}_{ds_root}'
            time = np.linspace(start_year,end_year,num=len(myVars[ds_name][var]) )
            ax.plot(time,myVars[ds_name][var],label=prof,color=prof_dict[prof])

        ctrl_ds_name = f'{co2_scen}_ctrl_{start_year}_{end_year}'
        time = np.linspace(start_year,end_year,num=len(myVars[ctrl_ds_name][var]) )
        ax.plot(time,myVars[ctrl_ds_name][var],label='control',color='k')

        ax.set_xlabel("Time (Years)")
        if var == "ave_hfds":
            ax.set_ylabel("Flux (W/m$^2$)")
        else:
            ax.set_ylabel("Temperature ($\degree$C)")
            
        ax.set_xlim(start_year-1,end_year)
        ax.set_ylim(ylimits)
        ax.legend(loc=leg_loc,ncols=leg_ncols)
        ax.grid("both")
    
        ax.minorticks_on()
        ax.grid(which='major', linestyle='-', linewidth='0.5', color='gray')

        if co2_scen == 'const':
            title_str = f"Const CO2 {power_strings[pow_idx]} Cases\n"
            fig_name = f"const_{power}"
            
        elif co2_scen == 'doub':
            title_str = f"1pct2xCO2 {power_strings[pow_idx]} Cases\n"
            fig_name = f"2xCO2_{power}"
            
        elif co2_scen == 'quad':
            title_str = f"1pct4xCO2 {power_strings[pow_idx]} Cases\n"
            fig_name = f"4xCO2_{power}"

        ax.set_title(title_str+title_suff)
        
        if not os.path.exists(fig_dir):
            os.makedirs(fig_dir)
        
        if fig_pref != None:
            plt.savefig(fig_dir+f'{fig_pref}_{var}_{fig_name}_{start_year}_{end_year}.pdf', dpi=600, bbox_inches='tight')
        else:
            plt.savefig(fig_dir+f'{var}_{fig_name}_{start_year}_{end_year}.pdf', dpi=600, bbox_inches='tight')

## Mixing and radiative anomaly components

In [3]:
def plot_ts_mixing_diff(co2_scen,fig_dir,start_year,end_year,
                        anom_var="thetaoga",title_suff="Volume Mean Ocean Temperature Anomaly",fig_pref=None,
                     ylimits = [-0.1,1.2],
                     leg_loc = 'upper left',
                     leg_ncols = 1,
                       profiles = ['surf','therm','mid','bot'], 
                       power_inputs = ['0.1TW', '0.2TW', '0.5TW'], 
                       power_var_suff = ['0p1TW', '0p2TW', '0p5TW'], 
                       power_strings = ['0.1 TW', '0.2 TW', '0.5 TW']):
    """
    Function to plot mixing response for multiple CO2 scenarios.
    
    Inputs:
    co2_scen (str): one of ['doub','quad','all']
    fig_dir (str): directory path to save figure
    start_year (int): start year of avg period
    end_year (int): end year of avg period
    """

    prof_dict = {'surf': 'b',
                 'therm': 'm',
                 'mid': 'g',
                 'bot': 'r'}

    if co2_scen == 'all':
        mixing_leg = [Line2D([0], [0], color='b', lw=2),
                    Line2D([0], [0], color='m', lw=2),
                    Line2D([0], [0], color='g', lw=2),
                    Line2D([0], [0], color='r', lw=2),
                    Line2D([0], [0], linestyle='solid', lw=2, color='k'),
                    Line2D([0], [0], linestyle='dashed', lw=2, color='k'),
                    Line2D([0], [0], linestyle='dotted', lw=2, color='k')]
        
        mixing_leg_labels = ['surf', 'therm', 'mid', 'bot', 
                             'const CO2', '1pct2xCO2', '1pct4xCO2']

    elif co2_scen == 'doub':
        mixing_leg = [Line2D([0], [0], color='b', lw=2),
                    Line2D([0], [0], color='m', lw=2),
                    Line2D([0], [0], color='g', lw=2),
                    Line2D([0], [0], color='r', lw=2),
                    Line2D([0], [0], linestyle='solid', lw=2, color='k'),
                    Line2D([0], [0], linestyle='dashed', lw=2, color='k')]
        
        mixing_leg_labels = ['surf', 'therm', 'mid', 'bot', 
                             'const CO2', '1pct2xCO2']

    elif co2_scen == 'quad':
        mixing_leg = [Line2D([0], [0], color='b', lw=2),
                    Line2D([0], [0], color='m', lw=2),
                    Line2D([0], [0], color='g', lw=2),
                    Line2D([0], [0], color='r', lw=2),
                    Line2D([0], [0], linestyle='solid', lw=2, color='k'),
                    Line2D([0], [0], linestyle='dotted', lw=2, color='k')]
        
        mixing_leg_labels = ['surf', 'therm', 'mid', 'bot', 
                             'const CO2', '1pct4xCO2']

    # separate plot for each power input
    for pow_idx, power in enumerate(power_var_suff):
        fig, ax = plt.subplots(figsize=(6,3))

        # Add a horizontal line at y=0
        ax.axhline(y=0, color='black', linestyle='-', linewidth=1)
        
        ds_root = f'{power}_{start_year}_{end_year}_diff'
        for prof in profiles:
            const_ds_name = f'const_{prof}_{ds_root}'
            doub_ds_name = f'doub_{prof}_{ds_root}_2xctrl'
            quad_ds_name = f'quad_{prof}_{ds_root}_4xctrl'

            const_anom = myVars[const_ds_name][anom_var]
            time = np.linspace(start_year,end_year,num=len(const_anom))
            ax.plot(time,const_anom,color=prof_dict[prof])
            
            if co2_scen == 'doub' or co2_scen == 'all':
                ax.plot(time,myVars[doub_ds_name][anom_var],linestyle='dashed',color=prof_dict[prof])
            elif co2_scen == 'quad' or co2_scen == 'all':
                ax.plot(time,myVars[quad_ds_name][anom_var],linestyle='dotted',color=prof_dict[prof])
            

        ax.set_xlabel("Time (Years)")
        ax.set_ylabel("Temperature Anomaly ($\degree$C)")
        ax.set_xlim(start_year-1,end_year)
        ax.set_ylim(ylimits)

        ax.legend(mixing_leg, mixing_leg_labels, loc=leg_loc, fontsize=10, ncol = leg_ncols, labelspacing=0.1)
        
        ax.grid("both")
        ax.minorticks_on()
        ax.grid(which='major', linestyle='-', linewidth='0.5', color='gray')

        title_str = f"Mixing $\Delta T$ Comparison: {power_strings[pow_idx]} Cases\n"
        
        if co2_scen == 'doub':
            fig_name = f"const_2xCO2_mixing_{power}"
        elif diff_type == 'quad':
            fig_name = f"const_4xCO2_mixing_{power}"
        elif diff_type == 'all':
            fig_name = f"all_mixing_{power}"
            
        ax.set_title(title_str+title_suff)
        
        if not os.path.exists(fig_dir):
            os.makedirs(fig_dir)

        if fig_pref != None:
            plt.savefig(fig_dir+f'{fig_pref}_{anom_var}_anom_{fig_name}_{start_year}_{end_year}.pdf', dpi=600, bbox_inches='tight')
        else:
            plt.savefig(fig_dir+f'{anom_var}_anom_{fig_name}_{start_year}_{end_year}.pdf', dpi=600, bbox_inches='tight')

In [4]:
def plot_ts_rad_pow_diff(co2_scen,fig_dir,start_year,end_year,
                     anom_var="thetaoga",title_suff="Volume Mean Ocean Temperature Anomaly",fig_pref=None,
                     ylimits = [-0.1,1.2],
                     leg_loc = 'upper left',
                     leg_ncols = 1):
    """
    Function to plot radiative response for multiple power inputs.
    
    Inputs:
    co2_scen (str): one of ['doub','quad']
    fig_dir (str): directory path to save figure
    start_year (int): start year of avg period
    end_year (int): end year of avg period
    """

    power_inputs = ['0.1TW', '0.2TW', '0.5TW']
    power_var_suff = ['0p1TW', '0p2TW', '0p5TW']
    power_strings = ['0.1 TW', '0.2 TW', '0.5 TW']

    profiles = ['surf','therm','mid','bot']

    prof_dict = {'surf': 'b',
                 'therm': 'm',
                 'mid': 'g',
                 'bot': 'r'}

    rad_leg = [Line2D([0], [0], color='b', lw=2),
                Line2D([0], [0], color='m', lw=2),
                Line2D([0], [0], color='g', lw=2),
                Line2D([0], [0], color='r', lw=2),
                Line2D([0], [0], linestyle='solid', lw=2, color='k'),
                Line2D([0], [0], linestyle='dashed', lw=2, color='k'),
                Line2D([0], [0], linestyle='dotted', lw=2, color='k')]
    
    rad_leg_labels = ['surf', 'therm', 'mid', 'bot', 
                         '0.1 TW', '0.2 TW', '0.5 TW']

    fig, ax = plt.subplots(figsize=(6,3))

    # Add a horizontal line at y=0
    ax.axhline(y=0, color='black', linestyle='-', linewidth=1)

    power_line_types = ['solid','dashed','dotted']
    
    for pow_idx, power in enumerate(power_var_suff):
        ds_root = f'{power}_{start_year}_{end_year}_diff'
        
        for prof in profiles:
            if co2_scen == 'doub':
                ds_name = f'doub_{prof}_{ds_root}_1860'
            elif co2_scen == 'quad':
                ds_name = f'quad_{prof}_{ds_root}_1860'
    
            rad_anom = myVars[ds_name][anom_var]
            time = np.linspace(start_year,end_year,num=len(rad_anom))
            ax.plot(time,rad_anom,color=prof_dict[prof],linestyle=power_line_types[pow_idx])            

    ax.set_xlabel("Time (Years)")
    ax.set_ylabel("Temperature Anomaly ($\degree$C)")
    ax.set_xlim(start_year-1,end_year)
    ax.set_ylim(ylimits)

    ax.legend(rad_leg, rad_leg_labels, loc=leg_loc, fontsize=10, ncol = leg_ncols, labelspacing=0.1)
    
    ax.grid("both")
    ax.minorticks_on()
    ax.grid(which='major', linestyle='-', linewidth='0.5', color='gray')
    
    if co2_scen == 'doub':
        title_str = f"1pct2xCO2 Radiative $\Delta T$ Comparison\n"
        fig_name = f"2xCO2_rad"
    elif diff_type == 'quad':
        title_str = f"1pct4xCO2 Radiative $\Delta T$ Comparison\n"
        fig_name = f"4xCO2_rad"
        
    ax.set_title(title_str+title_suff)
    
    if not os.path.exists(fig_dir):
        os.makedirs(fig_dir)
    
    if fig_pref != None:
        plt.savefig(fig_dir+f'{fig_pref}_{anom_var}_anom_{fig_name}_{start_year}_{end_year}.pdf', dpi=600, bbox_inches='tight')
    else:
        plt.savefig(fig_dir+f'{anom_var}_anom_{fig_name}_{start_year}_{end_year}.pdf', dpi=600, bbox_inches='tight')

In [5]:
def plot_ts_mix_pow_diff(co2_scen,fig_dir,start_year,end_year,
                     anom_var="thetaoga",title_suff="Volume Mean Ocean Temperature Anomaly",fig_pref=None,
                     ylimits = [-0.1,1.2],
                     leg_loc = 'upper left',
                     leg_ncols = 1,
                         power_var_suff = ['0p1TW', '0p2TW', '0p5TW'],
                         power_strings = ['0.1 TW', '0.2 TW', '0.5 TW']):
    """
    Function to plot mixing response for multiple power inputs.
    
    Inputs:
    co2_scen (str): one of ['const','doub','quad']
    fig_dir (str): directory path to save figure
    start_year (int): start year of avg period
    end_year (int): end year of avg period
    """

    profiles = ['surf','therm','mid','bot']

    prof_dict = {'surf': 'b',
                 'therm': 'm',
                 'mid': 'g',
                 'bot': 'r'}

    # Define custom legend entries
    mix_leg_1 = [  # First column (4 labels)
        Line2D([0], [0], color='b', lw=2),  # surf
        Line2D([0], [0], color='m', lw=2),  # therm
        Line2D([0], [0], color='g', lw=2),  # mid
        Line2D([0], [0], color='r', lw=2)  # bot
    ]
    labels_1 = ['surf', 'therm', 'mid', 'bot']
    
    mix_leg_2 = [  # Second column (3 labels)
        Line2D([0], [0], linestyle='solid', lw=2, color='k'),
        Line2D([0], [0], linestyle='dashed', lw=2, color='k'),
        Line2D([0], [0], linestyle='dotted', lw=2, color='k')
    ]
    labels_2 = power_strings

    mix_leg_3 = [Line2D([0], [0], alpha=0.6, lw=4, color='tab:gray')]

    if co2_scen == 'const' or co2_scen == 'doub':
        labels_3 = ['ctrl 1pct2xCO2']
    elif co2_scen == 'quad':
        labels_3 = ['ctrl 1pct4xCO2']

    fig, ax = plt.subplots(figsize=(6,3))

    # Add a horizontal line at y=0
    ax.axhline(y=0, color='black', linestyle='-', linewidth=1)

    power_line_types = ['solid','dashed','dotted']
    
    for pow_idx, power in enumerate(power_var_suff):
        ds_root = f'{power}_{start_year}_{end_year}_diff'
        
        for prof in profiles:
            if co2_scen == 'const':
                ds_name = f'const_{prof}_{ds_root}'
            elif co2_scen == 'doub':
                ds_name = f'doub_{prof}_{ds_root}_2xctrl'
            elif co2_scen == 'quad':
                ds_name = f'quad_{prof}_{ds_root}_4xctrl'
    
            mix_anom = myVars[ds_name][anom_var]
            time = np.linspace(start_year,end_year,num=len(mix_anom))
            ax.plot(time,mix_anom,color=prof_dict[prof],linestyle=power_line_types[pow_idx])

    if co2_scen == 'const' or co2_scen == 'doub':
        ctrl_rad_name = f'doub_ctrl_{start_year}_{end_year}_diff'
    elif co2_scen == 'quad':
        ctrl_rad_name = f'quad_ctrl_{start_year}_{end_year}_diff_const_ctrl'
        
    ctrl_rad_anom = myVars[ctrl_rad_name][anom_var]
    ax.plot(time,ctrl_rad_anom,color='tab:gray',alpha=0.6,lw=4)

    ax.set_xlabel("Time (Years)")
    ax.set_ylabel("Temperature Anomaly ($\degree$C)")
    ax.set_xlim(start_year-1,end_year)
    ax.set_ylim(ylimits)

    # ax.legend(mix_leg, mix_leg_labels, loc=leg_loc, fontsize=10, ncol = leg_ncols, labelspacing=0.1)

    # if leg_loc == 'upper left':
    legend1 = ax.legend(
    mix_leg_1, labels_1,
    loc=leg_loc,
    fontsize=10, labelspacing=0.1,
    bbox_to_anchor=(0.0, 1.0),  # Adjust position as needed
    frameon=True
    )
    # Second legend (2 labels, positioned below the first)
    legend2 = ax.legend(
        mix_leg_2, labels_2,
        loc=leg_loc,
        fontsize=10, labelspacing=0.1,
        bbox_to_anchor=(0.2, 1.0),  # Adjust position as needed
        frameon=True
    )
    # Second legend (2 labels, positioned below the first)
    legend3 = ax.legend(
        mix_leg_3, labels_3,
        loc=leg_loc,
        fontsize=10, labelspacing=0.1,
        bbox_to_anchor=(0.415, 1.0),  # Adjust position as needed
        frameon=True
    )
    
    # Add the first legend back to the axis
    ax.add_artist(legend1)
    ax.add_artist(legend2)
    
    ax.grid("both")
    ax.minorticks_on()
    ax.grid(which='major', linestyle='-', linewidth='0.5', color='gray')

    if co2_scen == 'const':
        title_str = f"Const $\Delta T$ Comparison\n"
        fig_name = f"const_mix"
    elif co2_scen == 'doub':
        title_str = f"1pct2xCO2 Mixing $\Delta T$ Comparison\n"
        fig_name = f"2xCO2_mix"
    elif co2_scen == 'quad':
        title_str = f"1pct4xCO2 Mixing $\Delta T$ Comparison\n"
        fig_name = f"4xCO2_mix"
        
    ax.set_title(title_str+title_suff)
    
    if not os.path.exists(fig_dir):
        os.makedirs(fig_dir)
    
    if fig_pref != None:
        plt.savefig(fig_dir+f'{fig_pref}_{anom_var}_anom_{fig_name}_{start_year}_{end_year}.pdf', dpi=600, bbox_inches='tight')
    else:
        plt.savefig(fig_dir+f'{anom_var}_anom_{fig_name}_{start_year}_{end_year}.pdf', dpi=600, bbox_inches='tight')

## Linearity of mixing and radiative anomaly components

In [6]:
def plot_lin_ts_diff(co2_scen,fig_dir,start_year,end_year,anom_var="thetaoga",title_suff="Volume Mean Ocean Temperature Anomaly",fig_pref=None,
                 ylimits = [-0.1,1.2],
                 leg_loc = 'upper left',
                       profiles = ['surf','therm','mid','bot'], 
                       power_inputs = ['0.1TW', '0.2TW', '0.5TW'], 
                       power_var_suff = ['0p1TW', '0p2TW', '0p5TW'], 
                       power_strings = ['0.1 TW', '0.2 TW', '0.5 TW']):
    """
    Function to plot sum of mixing and radiative responses and the realized total response.
    
    Inputs:
    co2_scen (str): one of ['const','doub','quad']
    fig_dir (str): directory path to save figure
    start_year (int): start year of avg period
    end_year (int): end year of avg period
    """

    prof_dict = {'surf': 'b',
                 'therm': 'm',
                 'mid': 'g',
                 'bot': 'r'}

    # Define custom legend entries
    CO2_lin_leg_1 = [  # First column (4 labels)
        Line2D([0], [0], color='b', lw=2),  # surf
        Line2D([0], [0], color='m', lw=2),  # therm
        Line2D([0], [0], color='g', lw=2),  # mid
        Line2D([0], [0], color='r', lw=2),  # bot
    ]
    labels_1 = ['surf', 'therm', 'mid', 'bot']
    
    CO2_lin_leg_2 = [  # Second column (2 labels)
        Line2D([0], [0], linestyle='solid', lw=2, color='k'),  # realized response
        Line2D([0], [0], linestyle='dashed', lw=2, color='k')   # CO2 + mixing
    ]
    labels_2 = ['realized response', 'CO2 + mixing']

    # separate plot for each power input
    for pow_idx, power in enumerate(power_var_suff):
        fig, ax = plt.subplots(figsize=(6,3))

        # Add a horizontal line at y=0
        ax.axhline(y=0, color='black', linestyle='-', linewidth=1)
        
        ds_root = f'{power}_{start_year}_{end_year}_diff'
        for prof in profiles:
            if co2_scen == 'doub':
                co2_ds_name = f'doub_{prof}_{ds_root}_1860'
                mixing_ds_name = f'doub_{prof}_{ds_root}_2xctrl'
                total_ds_name = f'doub_{prof}_{ds_root}_const_ctrl'
            elif co2_scen == 'quad':
                co2_ds_name = f'quad_{prof}_{ds_root}_1860'
                mixing_ds_name = f'quad_{prof}_{ds_root}_4xctrl'
                total_ds_name = f'quad_{prof}_{ds_root}_const_ctrl'

            co2_anom = myVars[co2_ds_name][anom_var]
            mixing_anom = myVars[mixing_ds_name][anom_var]
            total_anom = myVars[total_ds_name][anom_var]

            sum_anom = co2_anom + mixing_anom
            
            time = np.linspace(start_year,end_year,num=len(total_anom) )
            ax.plot(time,total_anom,color=prof_dict[prof])
            ax.plot(time,sum_anom,linestyle='--',color=prof_dict[prof])

        ax.set_xlabel("Time (Years)")
        ax.set_ylabel("Temperature Anomaly ($\degree$C)")
        ax.set_xlim(start_year-1,end_year)
        ax.set_ylim(ylimits)
        # ax.legend(loc=leg_loc,ncols=leg_ncols)

        if leg_loc == 'upper left':
            legend1 = ax.legend(
            CO2_lin_leg_1, labels_1,
            loc=leg_loc,
            fontsize=10, labelspacing=0.1,
            bbox_to_anchor=(0.0, 0.82),  # Adjust position as needed
            frameon=True
            )
            # Second legend (2 labels, positioned below the first)
            legend2 = ax.legend(
                CO2_lin_leg_2, labels_2,
                loc=leg_loc,
                fontsize=10, labelspacing=0.1,
                bbox_to_anchor=(0.0, 1.0),  # Adjust position as needed
                frameon=True
            )
        elif leg_loc == 'lower right':
            legend1 = ax.legend(
            CO2_lin_leg_1, labels_1,
            loc=leg_loc,
            fontsize=10, labelspacing=0.1,
            bbox_to_anchor=(1.0, 0.16),  # Adjust position as needed
            frameon=True
            )
            # Second legend (2 labels, positioned below the first)
            legend2 = ax.legend(
                CO2_lin_leg_2, labels_2,
                loc=leg_loc,
                fontsize=10, labelspacing=0.1,
                bbox_to_anchor=(1.0, 0.0),  # Adjust position as needed
                frameon=True
            )
        
        # Add the first legend back to the axis
        ax.add_artist(legend1)
        
        ax.grid("both")
        ax.minorticks_on()
        ax.grid(which='major', linestyle='-', linewidth='0.5', color='gray')
            
        if co2_scen == 'doub':
            title_str = f"1pct2xCO2 Total $\Delta T$ Comparison: {power_strings[pow_idx]} Cases\n"
            fig_name = f"2xCO2_lin_{power}"
            
        elif diff_type == 'quad':
            title_str = f"1pct4xCO2 Total $\Delta T$ Comparison: {power_strings[pow_idx]} Cases\n"
            fig_name = f"4xCO2_lin_{power}"
            
        ax.set_title(title_str+title_suff)
        
        if not os.path.exists(fig_dir):
            os.makedirs(fig_dir)
        
        if fig_pref != None:
            plt.savefig(fig_dir+f'{fig_pref}_{anom_var}_anom_{fig_name}_{start_year}_{end_year}.pdf', dpi=600, bbox_inches='tight')
        else:
            plt.savefig(fig_dir+f'{anom_var}_anom_{fig_name}_{start_year}_{end_year}.pdf', dpi=600, bbox_inches='tight')

# Atmos monthly variables (requires horizontal averaging)

## Regular anomaly and mean

In [7]:
def plot_atmos_ts_diff_2d(diff_type,fig_dir,start_year,end_year,anom_var="olr",title_suff="Mean OLR Anomaly",fig_pref=None,
                 ylimits = [-0.5,0.5],
                 leg_loc = 'upper left',
                 leg_ncols = 1,
                       profiles = ['surf','therm','mid','bot'], 
                       power_inputs = ['0.1TW', '0.2TW', '0.5TW'], 
                       power_var_suff = ['0p1TW', '0p2TW', '0p5TW'], 
                       power_strings = ['0.1 TW', '0.2 TW', '0.5 TW']):
    """
    Function to plot anomaly over time from time series data for a particular CO2 scenario (each power input on separate plot).
    
    Inputs:
    diff_type (str): one of
                    ['const-1860ctrl',
                    'doub-1860exp','doub-2xctrl','doub-1860ctrl',
                    'quad-1860exp','quad-4xctrl','quad-1860ctrl']
    fig_dir (str): directory path to save figure
    start_year (int): start year of avg period
    end_year (int): end year of avg period
    """

    prof_dict = {'surf': 'b',
                 'therm': 'm',
                 'mid': 'g',
                 'bot': 'r'}

    # separate plot for each power input
    for pow_idx, power in enumerate(power_var_suff):
        fig, ax = plt.subplots(figsize=(6,3))

        # Add a horizontal line at y=0
        ax.axhline(y=0, color='black', linestyle='-', linewidth=1)
        
        ds_root = f'{power}_{start_year}_{end_year}_diff'
        for prof in profiles:
            if diff_type == 'const-1860ctrl':
                ds_name = f'const_{prof}_{ds_root}'
            elif diff_type == 'doub-1860exp':
                ds_name = f'doub_{prof}_{ds_root}_1860'
            elif diff_type == 'doub-2xctrl':
                ds_name = f'doub_{prof}_{ds_root}_2xctrl'
            elif diff_type == 'doub-1860ctrl':
                ds_name = f'doub_{prof}_{ds_root}_const_ctrl'
            elif diff_type == 'quad-1860exp':
                ds_name = f'quad_{prof}_{ds_root}_1860'
            elif diff_type == 'quad-4xctrl':
                ds_name = f'quad_{prof}_{ds_root}_4xctrl'
            elif diff_type == 'quad-1860ctrl':
                ds_name = f'quad_{prof}_{ds_root}_const_ctrl'

            if anom_var == 'EEI':
                myVars[ds_name]['EEI'] = myVars[ds_name]['swdn_toa'] - myVars[ds_name]['swup_toa'] - myVars[ds_name]['olr']
                global_mean = atmos_horiz_mean(myVars[ds_name]['EEI'],myVars[ds_name])
            else:
                global_mean = atmos_horiz_mean(myVars[ds_name][anom_var],myVars[ds_name])

            time = np.linspace(start_year,end_year,num=len(global_mean) )
            ax.plot(time,global_mean,label=prof,color=prof_dict[prof])

        if diff_type == 'doub-1860exp' or diff_type == 'doub-1860ctrl':
            # plotting control difference wrt 1860 ctrl
            ds_name = f'doub_ctrl_{start_year}_{end_year}_diff'
            if anom_var == 'EEI':
                myVars[ds_name]['EEI'] = myVars[ds_name]['swdn_toa'] - myVars[ds_name]['swup_toa'] - myVars[ds_name]['olr']
                global_mean = atmos_horiz_mean(myVars[ds_name]['EEI'],myVars[ds_name])
            else:
                global_mean = atmos_horiz_mean(myVars[ds_name][anom_var],myVars[ds_name])
                
            time = np.linspace(start_year,end_year,num=len(global_mean))
            ax.plot(time,global_mean,label='control',color='k')

        ax.set_xlabel("Time (Years)")
        ax.set_ylabel("Flux Anomaly (W/m$^2$)")
        ax.set_xlim(start_year-1,end_year)
        ax.set_ylim(ylimits)
        ax.legend(loc=leg_loc,ncols=leg_ncols)
        ax.grid("both")
    
        ax.minorticks_on()
        ax.grid(which='major', linestyle='-', linewidth='0.5', color='gray')

        if diff_type == 'const-1860ctrl':
            title_str = f"Const CO2 $\Delta Q$: {power_strings[pow_idx]} Cases\n"
            fig_name = f"const_{power}"
            
        elif diff_type == 'doub-1860exp':
            title_str = f"1pct2xCO2 Radiative $\Delta Q$: {power_strings[pow_idx]} Cases\n"
            fig_name = f"2xCO2-const_{power}"
            
        elif diff_type == 'doub-2xctrl':
            title_str = f"1pct2xCO2 Mixing $\Delta Q$: {power_strings[pow_idx]} Cases\n"
            fig_name = f"2xCO2-2xctrl_{power}"
            
        elif diff_type == 'doub-1860ctrl':
            title_str = f"1pct2xCO2 Total $\Delta Q$: {power_strings[pow_idx]} Cases\n"
            fig_name = f"2xCO2-const-ctrl_{power}"
            
        elif diff_type == 'quad-1860exp':
            title_str = f"1pct4xCO2 Radiative $\Delta Q$: {power_strings[pow_idx]} Cases\n"
            fig_name = f"4xCO2-const_{power}"
            
        elif diff_type == 'quad-4xctrl':
            title_str = f"1pct4xCO2 Mixing $\Delta Q$: {power_strings[pow_idx]} Cases\n"
            fig_name = f"4xCO2-4xctrl_{power}"
            
        elif diff_type == 'quad-1860ctrl':
            title_str = f"1pct4xCO2 Total $\Delta Q$: {power_strings[pow_idx]} Cases\n"
            fig_name = f"4xCO2-const-ctrl_{power}"

        ax.set_title(title_str+title_suff)
        
        if not os.path.exists(fig_dir):
            os.makedirs(fig_dir)

        if fig_pref != None:
            plt.savefig(fig_dir+f'{fig_pref}_{anom_var}_anom_{fig_name}_{start_year}_{end_year}.pdf', dpi=600, bbox_inches='tight')
        else:
            plt.savefig(fig_dir+f'{anom_var}_anom_{fig_name}_{start_year}_{end_year}.pdf', dpi=600, bbox_inches='tight')

In [8]:
def plot_atmos_ts_2d(co2_scen,fig_dir,start_year,end_year,var="olr",title_suff="Mean OLR",fig_pref=None,
                 ylimits = [3.0,5.0],
                 leg_loc = 'upper left',
                 leg_ncols = 1,
                       profiles = ['surf','therm','mid','bot'], 
                       power_inputs = ['0.1TW', '0.2TW', '0.5TW'], 
                       power_var_suff = ['0p1TW', '0p2TW', '0p5TW'], 
                       power_strings = ['0.1 TW', '0.2 TW', '0.5 TW']):
    """
    Function to plot raw time series variable (not anomaly).
    
    Inputs:
    co2_scen (str): one of ['const','doub','quad']
    fig_dir (str): directory path to save figure
    start_year (int): start year of avg period
    end_year (int): end year of avg period
    """

    prof_dict = {'surf': 'b',
                 'therm': 'm',
                 'mid': 'g',
                 'bot': 'r'}

    # separate plot for each power input
    for pow_idx, power in enumerate(power_var_suff):
        fig, ax = plt.subplots(figsize=(6,3))

        # Add a horizontal line at y=0
        ax.axhline(y=0, color='black', linestyle='-', linewidth=1)
        
        ds_root = f'{power}_{start_year}_{end_year}'
        for prof in profiles:
            ds_name = f'{co2_scen}_{prof}_{ds_root}'

            if var == 'EEI':
                myVars[ds_name]['EEI'] = myVars[ds_name]['swdn_toa'] - myVars[ds_name]['swup_toa'] - myVars[ds_name]['olr']
                global_mean = atmos_horiz_mean(myVars[ds_name]['EEI'],myVars[ds_name])
            else:
                global_mean = atmos_horiz_mean(myVars[ds_name][var],myVars[ds_name])
                
            time = np.linspace(start_year,end_year,num=len(global_mean) )
            ax.plot(time,global_mean,label=prof,color=prof_dict[prof])

        # control
        ds_name = f'{co2_scen}_ctrl_{start_year}_{end_year}'

        if var == 'EEI':
            myVars[ds_name]['EEI'] = myVars[ds_name]['swdn_toa'] - myVars[ds_name]['swup_toa'] - myVars[ds_name]['olr']
            global_mean = atmos_horiz_mean(myVars[ds_name]['EEI'],myVars[ds_name])
        else:
            global_mean = atmos_horiz_mean(myVars[ds_name][var],myVars[ds_name])
        
        time = np.linspace(start_year,end_year,num=len(global_mean) )
        ax.plot(time,global_mean,label='control',color='k')
        ax.set_xlabel("Time (Years)")
        ax.set_ylabel("Flux Anomaly (W/m$^2$)")
        ax.set_xlim(start_year-1,end_year)
        ax.set_ylim(ylimits)
        ax.legend(loc=leg_loc,ncols=leg_ncols)
        ax.grid("both")
    
        ax.minorticks_on()
        ax.grid(which='major', linestyle='-', linewidth='0.5', color='gray')

        if co2_scen == 'const':
            title_str = f"Const CO2 {power_strings[pow_idx]} Cases\n"
            fig_name = f"const_{power}"
            
        elif co2_scen == 'doub':
            title_str = f"1pct2xCO2 {power_strings[pow_idx]} Cases\n"
            fig_name = f"2xCO2_{power}"
            
        elif co2_scen == 'quad':
            title_str = f"1pct4xCO2 {power_strings[pow_idx]} Cases\n"
            fig_name = f"4xCO2_{power}"

        ax.set_title(title_str+title_suff)
        
        if not os.path.exists(fig_dir):
            os.makedirs(fig_dir)
        
        if fig_pref != None:
            plt.savefig(fig_dir+f'{fig_pref}_{var}_{fig_name}_{start_year}_{end_year}.pdf', dpi=600, bbox_inches='tight')
        else:
            plt.savefig(fig_dir+f'{var}_{fig_name}_{start_year}_{end_year}.pdf', dpi=600, bbox_inches='tight')